In [10]:
import os
import pandas as pd
import numpy as np
import boto3

In [2]:
df = pd.read_excel('./data/final_output_with_images.xlsx')
df['path'] = df['image_url'].apply(lambda x: x.split('com/')[1])
print(f"Total {df['id'].count()} images. {df['path'].nunique()} of them are unique")

Total 28397 images. 27388 of them are unique


In [3]:
# Look at the head
df.sort_values(by = 'id', inplace= True)
df.head()

,id,verification,phone,date_taken,date_uploaded,lat,lon,picked up,address,total_litter,...,batteries,balloons.1,life_buoy,other,custom_tag_1,custom_tag_2,custom_tag_3,link,image_url,path
7365,4222,2,SAMSUNG-SM-J727A,12/12/17 11:27,1/25/18 17:59,29.456944,-98.422500,No,"Warrior and Family Support Center, George Beac...",4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://openlittermap.com/global?lat=29.456944...,https://olm-s3.s3.eu-west-1.amazonaws.com/112/...,112/Pftq4550WzgZA5euhgIEsLHGs63tXSHAqCP07WKl.jpeg
7366,4223,2,SAMSUNG-SM-J727A,12/11/17 15:04,1/25/18 17:59,29.565833,-98.593889,No,"Silicon Drive, San Antonio, Bexar County, Texa...",3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://openlittermap.com/global?lat=29.565833...,https://olm-s3.s3.eu-west-1.amazonaws.com/112/...,112/HKXafwXvrpBQjoWC9aPtSY8lTGcdoZVlPItqNQkK.jpeg
7367,4227,2,Unknown,1/25/18 18:24,1/25/18 18:24,29.455317,-98.424719,No,"Binz-Engleman Road, San Antonio, Bexar County,...",5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://openlittermap.com/global?lat=29.455316...,https://olm-s3.s3.eu-west-1.amazonaws.com/112/...,112/69M58FGWNVp1ZTObF0YmkMfGNEz1nUu1B2MSI8Gt.jpeg
7368,4228,2,Unknown,1/25/18 18:25,1/25/18 18:25,29.455422,-98.424697,No,"3746, Binz-Engleman Road, San Antonio, Bexar C...",2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://openlittermap.com/global?lat=29.455422...,https://olm-s3.s3.eu-west-1.amazonaws.com/112/...,112/Da9HiIUxqaxY7WeKi2INrzgTGzLGTvAl5HBhIEfn.jpeg
7369,4229,2,Unknown,1/25/18 18:25,1/25/18 18:25,29.457278,-98.422442,No,"Warrior and Family Support Center, George Beac...",5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://openlittermap.com/global?lat=29.457277...,https://olm-s3.s3.eu-west-1.amazonaws.com/112/...,112/pUY1im8pNbPUAceAv17T5SCvmwK8KIAmVatpCN2o.jpeg


In [4]:
# Create a boolean mask to identify duplicate rows based on 'path' column
duplicates = df.duplicated('path', keep=False)

# Filter the original dataframe to keep only the duplicate rows, show the head of duplicates
duplicates_df = df[duplicates]
duplicates_df.head()

,id,verification,phone,date_taken,date_uploaded,lat,lon,picked up,address,total_litter,...,batteries,balloons.1,life_buoy,other,custom_tag_1,custom_tag_2,custom_tag_3,link,image_url,path
1362,46315,2,Mobile app v1.01,9/19/19 12:16,9/26/19 19:08,40.714455,-74.002739,No,"Triumph of the Human Spirit, Civic Center, New...",2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://openlittermap.com/global?lat=40.714455...,https://olm-s3.s3.eu-west-1.amazonaws.com/2019...,2019/09/19/IMG_7939.HEIC
1363,46323,2,Mobile app v1.01,9/19/19 12:14,9/26/19 19:09,40.715150,-74.002717,No,"Thomas Paine Park, Civic Center, New York, New...",2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://openlittermap.com/global?lat=40.71515&...,https://olm-s3.s3.eu-west-1.amazonaws.com/2019...,2019/09/19/IMG_7931.HEIC
1367,46434,2,Mobile app v1.01,9/19/19 12:16,10/1/19 19:34,40.714455,-74.002739,No,"Triumph of the Human Spirit, Civic Center, New...",2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://openlittermap.com/global?lat=40.714455...,https://olm-s3.s3.eu-west-1.amazonaws.com/2019...,2019/09/19/IMG_7939.HEIC
1374,46442,2,Mobile app v1.01,9/19/19 12:14,10/1/19 19:34,40.715150,-74.002717,No,"Thomas Paine Park, Civic Center, New York, New...",1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://openlittermap.com/global?lat=40.71515&...,https://olm-s3.s3.eu-west-1.amazonaws.com/2019...,2019/09/19/IMG_7931.HEIC
5055,47370,2,SM-G950U,9/10/19 3:56,10/26/19 14:54,26.462191,-80.057404,No,"26, N Ocean Blvd, Delray Beach, Palm Beach Cou...",3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://openlittermap.com/global?lat=26.462190...,https://olm-s3.s3.eu-west-1.amazonaws.com/2019...,2019/09/10/29963.png


In [5]:
# Remove all duplicate paths, also get rid of the coluymns we do not need to copy the images
df = df.groupby('path', as_index=False)[['id','image_url']].first()

# Look at the head again to confirm we did not mess anything up
df.sort_values(by = 'id', inplace= True)
df.head()

,path,id,image_url
32,112/Pftq4550WzgZA5euhgIEsLHGs63tXSHAqCP07WKl.jpeg,4222,https://olm-s3.s3.eu-west-1.amazonaws.com/112/...
21,112/HKXafwXvrpBQjoWC9aPtSY8lTGcdoZVlPItqNQkK.jpeg,4223,https://olm-s3.s3.eu-west-1.amazonaws.com/112/...
6,112/69M58FGWNVp1ZTObF0YmkMfGNEz1nUu1B2MSI8Gt.jpeg,4227,https://olm-s3.s3.eu-west-1.amazonaws.com/112/...
17,112/Da9HiIUxqaxY7WeKi2INrzgTGzLGTvAl5HBhIEfn.jpeg,4228,https://olm-s3.s3.eu-west-1.amazonaws.com/112/...
76,112/pUY1im8pNbPUAceAv17T5SCvmwK8KIAmVatpCN2o.jpeg,4229,https://olm-s3.s3.eu-west-1.amazonaws.com/112/...


In [6]:
num_rows = 300
unique_indices = np.random.choice(df.index, size=num_rows, replace=False)
random_df = df.loc[unique_indices]
random_df.head()

,path,id,image_url
19473,2022/02/28/JJfsPw1WlTpyavs7r4XyFlqhppdzrvkkU6l...,266432,https://olm-s3.s3.eu-west-1.amazonaws.com/2022...
19388,2022/02/25/rdBEsUNHk6jXbLxtSzML5aIBotEBhlWY5yD...,265646,https://olm-s3.s3.eu-west-1.amazonaws.com/2022...
23988,2022/12/08/3GfesBlQXKEKJelobWYE8JNQbkrDknpNPam...,360178,https://olm-s3.s3.eu-west-1.amazonaws.com/2022...
2328,2020/05/09/IMG_1211.HEIC,75655,https://olm-s3.s3.eu-west-1.amazonaws.com/2020...
18735,2022/02/17/Kg2EpAAu7R199EEy62kt8uFVCBjSHxWctgU...,263204,https://olm-s3.s3.eu-west-1.amazonaws.com/2022...


In [ ]:
# Copy all images from OLM s3 to my s3
for filename in random_df['path']:
    print(filename)
    command = f"aws s3 cp s3://olm-pics-s3/{filename} sample_images/{filename.split('/')[-1]}"
    os.system(command) 

In [11]:
# Create an S3 client
s3 = boto3.client('s3')

# Specify the bucket name
bucket_name = 'olm-pics-s3'

# Get the list of objects in the bucket
response = s3.list_objects_v2(Bucket=bucket_name)

# Create an empty list to store the file information
file_info = []

# Iterate through the objects and extract the relevant information
while response['IsTruncated']:
    for obj in response['Contents']:
        file_info.append({
            'Key': obj['Key'],
            'Size': obj['Size'],
            'LastModified': obj['LastModified']
        })
    token = response['NextContinuationToken']
    response = s3.list_objects_v2(Bucket=bucket_name, ContinuationToken=token)

for obj in response['Contents']:
    file_info.append({
        'Key': obj['Key'],
        'Size': obj['Size'],
        'LastModified': obj['LastModified']
    })

# Create a Pandas DataFrame from the file information
df_names = pd.DataFrame(file_info)
df_names


,Key,Size,LastModified
0,1/tTyxAD9TsRwRm6yHD8TZrmin76zbW08A9NXCaxz9.jpeg,678377,2024-06-04 22:40:15+00:00
1,112/18RfT51ZmG4CbSBoRLQKOSFB5J6p8Hc1XX3qosy6.jpeg,3203040,2024-06-04 22:52:35+00:00
2,112/2ObwUbTXkBbQKOmW0xXLhf6msJCqk9G4pQDyHzpd.jpeg,526727,2024-06-04 22:38:06+00:00
3,112/3pci4ANxHOGByqVp5xHuNUAjkJLUqnRIcBPTX237.jpeg,2363024,2024-06-04 22:52:39+00:00
4,112/3q5V6dsu75SItwjaXdJ7bvODcykTgpaemhDPbv0l.jpeg,3518964,2024-06-04 22:53:17+00:00
...,...,...,...
13170,796/e21Jf08oIqEH6wd7HwEWek1hxImtKkSZQLNV8Q9z.jpeg,4672916,2024-06-04 22:39:54+00:00
13171,796/g0bn4kyf6i5JA3go6GZIjeGfmdujSQBtU8ipYaMn.jpeg,3308917,2024-06-04 22:39:30+00:00
13172,796/mTNtBGV9iWisT7YyUbDF63btLjGf6ensiYeXImNq.jpeg,4557128,2024-06-04 22:39:36+00:00
13173,796/o5LSbZXxUrKNG7ytBBaGsUvnNcmo8FQdMRS6fhZV.jpeg,5028356,2024-06-04 22:39:23+00:00
